In [68]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import pandas as pd
import datetime
import uuid
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

In [2]:
main_url = 'http://cronica.diputados.gob.mx/DDebates/index.html'

In [81]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install()
                          #, options=options
                         )

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [4]:
driver.get(main_url)

In [5]:
def month_to_number(m):
    if m == 'ene':
        return(1)
    elif m == 'feb':
        return(2)
    elif m == 'mar':
        return(3)
    elif m == 'abr':
        return(4)
    elif m == 'may':
        return(5)
    elif m == 'jun':
        return(6)
    elif m == 'jul':
        return(7)
    elif m == 'ago':
        return(8)
    elif m == 'sep':
        return(9)
    elif m == 'oct':
        return(10)
    elif m == 'nov':
        return(11)
    elif m == 'dic':
        return(12)

def get_date_speech(year_session, link_speech, text_day):
    day_speech = re.findall('[0-9]+', text_day)[0]
    month_speech = month_to_number(link_speech.split('/')[-2:][0])
    return(datetime.date(int(year_session), int(month_speech), int(day_speech)))

def get_links_legislaturas(legislaturas):
    """
    Parametros:
    -----------
    legislatura: list of int numbers of legislatura 
    
    Returns:
    --------
    lista de diccionario con los a el htmls de los debates legislativos
    """
    all_data = []
    
    for legislatura in legislaturas:
        link_debates = 'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/index.html'
        
        driver.get(link_debates)
        
        links_sesiones = []
        type_sesion = []
        for h in driver.find_elements_by_css_selector('a'):
            links_sesiones.append(h.get_attribute('href'))
            type_sesion.append(re.sub('^[ ]+', '', h.text))
        
        
        for x, y in zip(links_sesiones, type_sesion):
            driver.get(x)
            
            for i in driver.find_elements_by_css_selector('a'):
                year_session = re.findall('[0-9]{4}', driver.find_element_by_css_selector('body > table:nth-child(3) > tbody > tr > td').text)[0]
                link_ = i.get_attribute('href')
                
                fecha_speech = get_date_speech(year_session, link_, i.text)
        
                temp = {}
                temp['id_debate'] = str(uuid.uuid3(uuid.NAMESPACE_URL,link_))
                temp['legislatura'] = legislatura
                temp['date_debate'] = fecha_speech
                temp['type_sesion'] = y
                temp['link_debate'] = link_
                all_data.append(temp)
    return(all_data)
    

In [6]:
htmls_speeches = get_links_legislaturas([63, 64])

In [70]:
df_debates = pd.DataFrame(htmls_speeches)

In [ ]:
def obtain_texto_debate(driver):
    contenido_debate = driver.find_elements_by_class_name('Contenido')[0].text
    
    dict_contenidos = {}
    for i in contenido_debate.split('\n\n\n'):
        try:
            name_tema = re.sub('\n', '', re.findall('^[A-Z].*\n',i)[0])
            dict_contenidos[name_tema] = re.sub(name_tema, '', i)
        except:
            continue
    
    main_text = []
    for x in dict_contenidos.values():
        main_text.append(x)
    return(re.sub('\n', ' ', ' '.join(main_text)))

In [82]:
driver.get('http://cronica.diputados.gob.mx/DDebates/63/1er/1P/Ord/sep/Sesion4.html')

In [97]:

def obtain_text_sesion(driver):
    try:
        links_volumenes = []
        for i in driver.find_elements_by_css_selector('table')[1].find_elements_by_css_selector('a'):
            if any(re.findall('Volúmen' ,i.text)):
                links_volumenes.append(i.get_attribute('href'))
            else:
                continue
        
        driver.get(links_volumenes[0])
        return(obtain_texto_debate(driver))
    except:
        return(obtain_texto_debate(driver))
        
            

In [96]:
case_volumnes(driver)

'  ASISTENCIA El Presidente diputado José de Jesús Zambrano Grijalva: Pido a la Secretaría haga del conocimiento de esta Presidencia el resultado del cómputo de asistencia de diputadas y diputados. La Secretaria diputada Ernestina Godoy Ramos:Con su venia, presidente. Se informa a la Presidencia que existen registrados previamente 434 diputadas y diputados; por lo tanto hay quórum. El Presidente diputado José de Jesús Zambrano Grijalva(a las 11:40 horas): Se abre la sesión.  El Presidente diputado José de Jesús Zambrano Grijalva: Consulte la Secretaría a la asamblea si se dispensa la lectura al orden del día. La Secretaria diputada Ernestina Godoy Ramos:Por instrucciones de la Presidencia, en votación económica se consulta a la asamblea si se dispensa la lectura al orden del día. Las diputadas y los diputados que estén por la afirmativa sírvanse manifestarlo. Las diputadas y los diputados que estén por la negativa sírvanse manifestarlo. Señor presidente, mayoría por la afirmativa.Se di

In [93]:
driver.get('http://cronica.diputados.gob.mx/DDebates/63/1er/1P/Ord/oct/Sesion13.html')

In [94]:
case_volumnes(driver)

'Presidencia del diputado José de Jesús Zambrano Grijalva\n\nASISTENCIA\nEl Presidente diputado José de Jesús Zambrano Grijalva: Pido a la Secretaría haga del conocimiento de esta Presidencia el resultado del cómputo de asistencia de diputadas y diputados.\nLa Secretaria diputada Ernestina Godoy Ramos:Con su venia, presidente. Se informa a la Presidencia que existen registrados previamente 434 diputadas y diputados; por lo tanto hay quórum.\nEl Presidente diputado José de Jesús Zambrano Grijalva(a las 11:40 horas): Se abre la sesión.\n\n\nORDEN DEL DÍA\nEl Presidente diputado José de Jesús Zambrano Grijalva: Consulte la Secretaría a la asamblea si se dispensa la lectura al orden del día.\nLa Secretaria diputada Ernestina Godoy Ramos:Por instrucciones de la Presidencia, en votación económica se consulta a la asamblea si se dispensa la lectura al orden del día. Las diputadas y los diputados que estén por la afirmativa sírvanse manifestarlo. Las diputadas y los diputados que estén por la 

In [98]:
text_congress = []
for i in tqdm(df_debates.link_debate):
    driver.get(i)
    try:
        text_congress.append(obtain_text_sesion(driver))
    except:
        print(i)

 45%|████▍     | 170/378 [32:43<22:15,  6.42s/it]  

http://cronica.diputados.gob.mx/DDebates/63/3er/1P/Ord/oct/Seseion18.html


 52%|█████▏    | 197/378 [37:38<10:57,  3.63s/it]  

http://cronica.diputados.gob.mx/DDebates/63/3er/2P/Ord/feb/Sesion2.html


 73%|███████▎  | 275/378 [51:16<05:53,  3.43s/it]  

http://cronica.diputados.gob.mx/DDebates/64/1er/1P/1Extra/ene/01L64A1E1022.html


100%|██████████| 378/378 [1:10:30<00:00, 11.19s/it]


In [99]:
len(text_congress)

375

In [74]:
driver.get(i)

In [16]:
pd.DataFrame(htmls_speeches)

,id_debate,legislatura,date_debate,type_sesion,link_debate
0,c0ceba86-4f50-3a1f-8907-e83005ee8720,63,2015-08-29,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
1,fb1799c9-00e9-3a85-b870-a54dbd24512a,63,2015-09-01,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
2,71979612-519d-32bd-ba03-5ce6c7b92e18,63,2015-09-03,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
3,37ea8705-c8f0-306d-91cf-6801203235a8,63,2015-09-08,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
4,a530b0c6-94a2-3b22-955f-5cb57544dea5,63,2015-09-10,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
...,...,...,...,...,...
373,b3f2f4ad-b4cb-3ef7-8c27-304f1f4565c4,64,2020-03-19,Segundo Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...
374,7f1c9c76-f4aa-3772-a37c-95d627b805d4,64,2020-03-19,Segundo Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...
375,a36388e6-7e70-3379-afbd-7560939c6952,64,2020-06-30,Primer Periodo Extraordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...
376,d570d6a4-9083-3293-8d59-49f1939659dd,64,2020-07-22,Segundo Periodo Extraordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...


In [17]:
driver.get(pd.DataFrame(htmls_speeches).link_debate[0])

In [34]:
def metadata_debate(driver):
    director_general_cronica_parlamentaria = driver.find_element_by_id('T1').text.split('\n')[-1:][0]
    presidente = driver.find_element_by_id('T2').text.split('\n')[-1:][0]
    director_diario_de_los_debates = driver.find_element_by_id('T3').text.split('\n')[-1:][0]
    sesion = driver.find_element_by_id('Sesion').text
    return(director_general_cronica_parlamentaria, presidente, director_diario_de_los_debates, sesion)

In [26]:
driver.find_element_by_id('T3').text.split('\n')[-1:][0]

'Luis Alfredo Mora Villagómez'

In [36]:
metadata_debate(driver)

('Gilberto Becerril Olivares',
 'Diputado José de Jesús Zambrano Grijalva',
 'Luis Alfredo Mora Villagómez',
 'Sesión No. 1')

In [59]:
def obtain_entradas_debates(driver):
    contenido_debate = driver.find_elements_by_class_name('Contenido')[0].text
    
    dict_contenidos = {}
    for i in contenido_debate.split('\n\n\n'):
        name_tema = re.sub('\n', '', re.findall('^[A-Z].*\n',i)[0])
        dict_contenidos[name_tema] = re.sub(name_tema, '', i)
    return(dict_contenidos)

In [77]:
def obtain_texto_debate(driver):
    contenido_debate = driver.find_elements_by_class_name('Contenido')[0].text
    
    dict_contenidos = {}
    for i in contenido_debate.split('\n\n\n'):
        try:
            name_tema = re.sub('\n', '', re.findall('^[A-Z].*\n',i)[0])
            dict_contenidos[name_tema] = re.sub(name_tema, '', i)
        except:
            continue
    
    main_text = []
    for x in dict_contenidos.values():
        main_text.append(x)
    return(re.sub('\n', ' ', ' '.join(main_text)))

In [76]:
driver.find_elements_by_class_name('Contenido')[0].text.split('\n\n\n')

['Presidencia del diputado José de Jesús Zambrano Grijalva',
 'ASISTENCIA\nEl Presidente diputado José de Jesús Zambrano Grijalva:  Pido a la Secretaría haga del conocimiento de esta Presidencia el resultado del cómputo de asistencia de diputadas y diputados.\nLa Secretaria diputada Ana Guadalupe Perea Santos: Se informa a la Presidencia que existen registrados previamente 462 diputadas y diputados. Por lo tanto hay quórum, señor presidente.\nEl Presidente diputado José de Jesús Zambrano Grijalva(a las 11:28 horas): Gracias. Se abre la sesión',
 'ORDEN DEL DÍA\nEl Presidente diputado José de Jesús Zambrano Grijalva: Consulte la Secretaría a la asamblea si se dispensa la lectura al orden del día.\nLa Secretaria diputada Ana Guadalupe Perea Santos: Por instrucciones de la Presidencia, en votación económica se consulta a la asamblea si se dispensa la lectura del orden del día. Las diputadas y los diputados que estén por la afirmativa sírvanse manifestarlo. Gracias. Las diputadas y los dip

In [43]:
driver.find_elements_by_class_name('Tema')[0].text

'INFORME DE LA SECRETARÍA GENERAL'

In [54]:
contenido_debate = driver.find_elements_by_class_name('Contenido')[0].text

In [56]:
dict_contenidos = {}
for i in contenido_debate.split('\n\n\n'):
    name_tema = re.sub('\n', '', re.findall('^[A-Z].*\n',i)[0])
    dict_contenidos[name_tema] = re.sub(name_tema, '', i)

In [58]:
dict_contenidos.keys()

dict_keys(['INFORME DE LA SECRETARÍA GENERAL', 'INTEGRACIÓN DE LA MESA DE DECANOS', 'DECLARACIÓN DE QUORUM', 'ORDEN DEL DÍA', 'PROTESTA CONSTITUCIONAL DE LA PRESIDENTA DE LA MESA DE DECANOS', 'PROTESTA CONSTITUCIONAL DE LAS DIPUTADAS Y LOS DIPUTADOS ELECTOS A LA SEXAGÉSIMA TERCERA LEGISLATURA', 'ELECCIÓN DE LOS INTEGRANTES DE LA MESA DIRECTIVA QUE FUNCIONARÁ DURANTE EL PRIMER AÑO DE EJERCICIO DE LA SEXAGÉSIMA TERCERA LEGISLATURA', 'TOMA DE PROTESTA DE LA MESA DIRECTIVA', 'DECLARACIÓN DE LA LEGAL CONSTITUCIÓN DE LA SEXAGÉSIMA TERCERA LEGISLATURA DE LA CÁMARA DE DIPUTADOS', 'MENSAJE DE LA PRESIDENCIA DE LA MESA DIRECTIVA', 'HIMNO NACIONAL', 'CITA'])

In [55]:
re.sub('\n', '', re.findall('^[A-Z].*\n', contenido_debate.split('\n\n\n')[0])[0])

'INFORME DE LA SECRETARÍA GENERAL'

In [50]:
contenido_debate.split('\n\n\n')[0]

'INFORME DE LA SECRETARÍA GENERAL\nEl secretario general Mauricio Farah Gebara(a las 11:31 horas): Honorable asamblea, señoras diputadas federales electas, señores diputados federales electos, de conformidad con lo dispuesto en el artículo 14 de la Ley Orgánica del Congreso General de los Estados Unidos Mexicanos y a efecto de la realización de esta sesión, entre los días 20 y 28 de agosto se registraron ante la Secretaría General de esta soberanía, 496 diputadas y diputados electos, es decir, la mayoría de sus integrantes.\nDe igual forma, la asistencia de los presentes el día de hoy atiende a lo dispuesto en el artículo 63 de la Constitución Política de los Estados Unidos Mexicanos.\nEn consecuencia, puede dar inicio la sesión constitutiva de la Cámara de Diputados a la LXIII Legislatura del Honorable Congreso de la Unión de los Estados Unidos Mexicanos conforme al siguiente orden del día.\nUno. Informe de la Secretaría General e integración de la Mesa de Decanos.\nDos. Declaración d